In [15]:
import re
from owlready2 import *
from pybliometrics.scopus import ScopusSearch
import pandas as pd

In [16]:
#load the ontology and perform reasoning
onto_new= "afo_upd"
new_world = owlready2.World()
onto = get_ontology("./ontologies/{}.owl".format(onto_new)).load()
sync_reasoner(onto) 

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp C:\Users\chern\anaconda3\envs\catnew\lib\site-packages\owlready2\hermit;C:\Users\chern\anaconda3\envs\catnew\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/chern/AppData/Local/Temp/tmpg2dbkojk
* Owlready2 * HermiT took 91.0519323348999 seconds
* Owlready * Reparenting property.AFX_0000974: {property.AFX_0000972, owl.ObjectProperty} => {property.AFX_0000972}
* Owlready * Reparenting core.narrower: {owl.ObjectProperty, core.narrowerTransitive} => {core.narrowerTransitive}
* Owlready * Reparenting obo.BFO_0000117: {owl.ObjectProperty, obo.BFO_0000051, obo.RO_0002222} => {obo.BFO_0000051, obo.RO_0002222}
* Owlready * Reparenting property.AFX_0002793: {owl.ObjectProperty, obo.BFO_0000051} => {obo.BFO_0000051}
* Owlready * Reparenting property.AFX_0002722: {owl.ObjectProperty, obo.RO_0002502} => {obo.RO_0002502}
* Owlready * Reparenting property.AFX_0001048: {owl.ObjectPr

In [3]:
#query function for reaction retrieval
def get_reaction(reac=None,doi=None,include_all=False):
    if doi!=None:
        doi= '"{}".}}'.format(doi)

    if reac==None and doi!=None: 
        sparqlstr="""    
            SELECT ?reacLabel
            WHERE{
                ?reaction rdf:type owl:NamedIndividual.
                ?reaction rdf:type ?type.
                ?type rdfs:subClassOf* obo:BFO_0000015.
                ?reaction rdfs:label ?reacLabel
                ?reaction afo:mentioned_in ?mention.
                ?mention afo:has_doi """ +doi
    elif reac==None and doi==None:
        sparqlstr="""    
            SELECT ?reacLabel ?doi
            WHERE{
                ?reaction rdf:type owl:NamedIndividual.
                ?reaction rdf:type ?type.
                ?type rdfs:subClassOf* obo:BFO_0000015.
                ?reaction rdfs:label ?reacLabel
                ?reaction afo:mentioned_in ?mention.
                ?mention afo:has_doi ?doi.}"""

    elif include_all==True:
        sparqlstr="""
            SELECT ?label ?doi
            WHERE{
                ?reaction rdf:type owl:NamedIndividual.
                ?reaction rdf:type ?type.
                ?type rdfs:subClassOf* obo:BFO_0000015.
                ?reaction rdfs:label ?label.
                FILTER regex(STR(?label),"hydroformylation","i").
                ?reaction afo:mentioned_in ?mention.
                ?mention afo:has_doi ?doi}"""
    else: 
        regex_reac='"{}"'.format(reac)
        sparqlstr="""
            SELECT ?doi
            WHERE{
                ?reaction rdf:type owl:NamedIndividual.
                ?reaction rdf:type ?type.
                ?type rdfs:subClassOf* obo:BFO_0000015.
                ?reaction rdfs:label """ +regex_reac+""".
                ?reaction afo:mentioned_in ?mention.
                ?mention afo:has_doi ?doi}""" 
            #BFO_0000015="process ";reactions are individuals of process subclasses
    

    sparqlstr= """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    PREFIX afo: <http://purl.allotrope.org/voc/afo/merged/REC/2023/03/merged-without-qudt-and-inferred#>
    PREFIX new:  <http://www.semanticweb.org/chern/ontologies/2023/10/new_onto.owl#>
    """+sparqlstr        
    try:
        reaction_list= list(default_world.sparql(sparqlstr))
    except:
        reaction_list= []
    return reaction_list   

In [6]:
#queries for reaction entities
doi = r'10.1016/1381-1169(96)00243-9'
list_reac_doi = get_reaction(reac=None,doi=None) #get list of all reactions and corresponding doi
#print(list_reac_doi)
list_reac = [*set([i[0] for i in list_reac_doi])]
print(list_reac)

['decarbonylation', 'hydrogenation', 'Hydroformylation', 'dissociation', 'hydroformylation', 'atmospheric hydroformylation', 'heterogeneous hydroformylation']


In [10]:
#get the list of all publications in ontology where same reactions as in input publication were mentioned
doi = r'10.1016/1381-1169(96)00243-9'
list_reac_doi = get_reaction(reac=None,doi=doi) #get list of all reactions mentioned in given doi (doi needs to be part of ontology)
same_reac_doi = []
for i in list_reac_doi:
    reac_doi = get_reaction(reac = i[0],doi = None)
    for c in reac_doi:
        if c not in same_reac_doi and c[0] != doi:
            same_reac_doi.append(c)#output example: [['10.1016/0304-5102(93)87113-m'],['10.1016/1381-1169(96)00243-9']]
print(list_reac_doi)
print(same_reac_doi)

[['hydroformylation'], ['atmospheric hydroformylation']]
[['10.1016/0304-5102(93)87113-m'], ['10.1021/acscatal.1c02014.s001'], ['10.1021/acscatal.1c04359'], ['10.1021/acscatal.7b00499.s001'], ['10.1021/jacs.1c09665.s001'], ['10.1016/s0920-5861(00)00261-3'], ['10.1016/j.apcata.2013.10.019'], ['10.1016/s1381-1169(97)00035-6'], ['10.1021/acs.iecr.9b03598.s001'], ['10.1021/acsami.0c21749.s001'], ['10.1021/acscatal.0c04684.s001'], ['10.1021/jacs.2c11075.s001']]


In [11]:
#get the list of publications which have "hydroformulation" reaction
list_reac_doi = get_reaction(reac="hydroformylation",doi=None)  #output example: [['10.1016/0304-5102(93)87113-m'],['10.1016/1381-1169(96)00243-9']]
print(list_reac_doi)

[['10.1016/0304-5102(93)87113-m'], ['10.1021/acscatal.1c02014.s001'], ['10.1021/acscatal.1c04359'], ['10.1021/acscatal.7b00499.s001'], ['10.1021/jacs.1c09665.s001'], ['10.1016/1381-1169(96)00243-9'], ['10.1016/s0920-5861(00)00261-3'], ['10.1016/j.apcata.2013.10.019'], ['10.1016/s1381-1169(97)00035-6'], ['10.1021/acs.iecr.9b03598.s001'], ['10.1021/acsami.0c21749.s001'], ['10.1021/acscatal.0c04684.s001'], ['10.1021/jacs.2c11075.s001']]


In [12]:
#get list of all publication which have "hydroformulation" and reactions which have "hydroformylation" in label
list_reac_doi = get_reaction(reac="hydroformylation",doi=None, include_all=True)
print(list_reac_doi)

[['hydroformylation', '10.1016/0304-5102(93)87113-m'], ['hydroformylation', '10.1021/acscatal.1c02014.s001'], ['hydroformylation', '10.1021/acscatal.1c04359'], ['hydroformylation', '10.1021/acscatal.7b00499.s001'], ['hydroformylation', '10.1021/jacs.1c09665.s001'], ['hydroformylation', '10.1016/1381-1169(96)00243-9'], ['hydroformylation', '10.1016/s0920-5861(00)00261-3'], ['hydroformylation', '10.1016/j.apcata.2013.10.019'], ['hydroformylation', '10.1016/s1381-1169(97)00035-6'], ['hydroformylation', '10.1021/acs.iecr.9b03598.s001'], ['hydroformylation', '10.1021/acsami.0c21749.s001'], ['hydroformylation', '10.1021/acscatal.0c04684.s001'], ['hydroformylation', '10.1021/jacs.2c11075.s001'], ['atmospheric hydroformylation', '10.1016/1381-1169(96)00243-9'], ['heterogeneous hydroformylation', '10.1021/acscatal.1c04359'], ['Hydroformylation', '10.1021/jacs.1c09665.s001']]


In [1]:
#query function for catalyst retrieval
def get_catalyst(cat=None,doi=None,restriction=None, include_all=False,add_support=False):
    if doi!=None:
        doi= '"{}".}}'.format(doi) 
        
    if restriction!=None:
        restr=''
    
    if cat ==None:
        sparqlstr="""
WHERE{
        {
                ?super_catalyst rdf:type role:AFRL_0000217.
        		?catalyst_e rdf:type owl:NamedIndividual.
		        ?catalyst_e rdf:type ?type.
        		?type rdfs:subClassOf* role:AFRL_0000217.
                ?catalyst_e rdfs:label ?catalyst.
          		FILTER NOT EXISTS {
        				FILTER regex(STR(?catalyst), "role").
        						}
        }
	UNION
    	{
                
        		?catalyst_full rdf:type owl:NamedIndividual.
        	 	?catalyst_full rdf:type ?type1.
        		?type1 rdfs:subClassOf* ?super_catalyst.
                ?super_catalyst rdfs:label 'chemical substance'.
                ?catalyst_e afo:catalytic_component_of ?catalyst_full.	
                ?catalyst_e rdfs:label ?catalyst.
                
        }
    """
    
    if include_all==True and cat ==None:
        
        sparqlstr=sparqlstr+"""
        UNION
             {
                ?catalyst_e rdf:type owl:NamedIndividual.
         	 	?catalyst_e obo:RO_0000087 ?cat_role.
                ?cat_role rdfs:label "catalyst role".
                ?super_catalyst rdf:type ?type2.
                ?catalyst_e rdfs:subClassOf ?type2.
                 FILTER NOT EXISTS {
  						
  						?catalyst_chem rdf:type ?type.
                 		?type rdfs:subClassOf* ?chem_substance.
                        ?chem_substance rdfs:label ?chemlabel.
                        FILTER regex(?chemlabel, 'chemical substance').
                        ?catalyst_e obo:RO_0000087 ?support_role.
                        ?support_role rdfs:label "support role".
                          }
                          
                 }
             """
    if add_support!=True:       
        sparqlstr=sparqlstr+"""
             ?catalyst_full afo:has_support_component ?support.
         """ 
        select= """SELECT ?super_catalyst ?catalyst_e ?catalyst ?catalyst_full ?doi ?support"""
    else:
        select= """SELECT ?super_catalyst ?catalyst_e ?catalyst ?catalyst_full ?doi"""
    if doi==None:
        sparqlstr=sparqlstr+"""
        	?catalyst_e afo:mentioned_in ?mention.
            ?mention afo:has_doi ?doi.}
            """
    else:
        sparqlstr=sparqlstr+"""
        	?catalyst_e afo:mentioned_in ?mention.
            ?mention afo:has_doi """+doi
        
    sparqlstr="""
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    PREFIX afo: <http://purl.allotrope.org/voc/afo/merged/REC/2023/03/merged-without-qudt-and-inferred#>
    PREFIX role: <http://purl.allotrope.org/ontologies/role#>
    PREFIX new:  <http://www.semanticweb.org/chern/ontologies/2023/10/new_onto.owl#>
    """+select+sparqlstr
    catalyst_list=list(default_world.sparql(sparqlstr))
    return catalyst_list


In [21]:
cat_list=get_catalyst(cat=None,doi=None,restriction=None, include_all=False,add_support=False)
print(cat_list)

In [22]:
print(cat_list)

[]


In [ ]:
def get_entList(list_type,doi=None): #list_type one of ['reactant','support','product']
    if doi== None:
        doi= '?doi'
    else:
        doi='new:"'+doi+'"'
    sparqlstr='''
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX obo: <http://purl.obolibrary.org/obo/>
    PREFIX afo: <http://purl.allotrope.org/voc/afo/merged/REC/2023/03/merged-without-qudt-and-inferred#>
    PREFIX new:  <http://www.semanticweb.org/chern/ontologies/2023/10/new_onto.owl#>
    SELECT ?ent ?entOtherName
    WHERE{
              ?ent rdf:type owl:NamedIndividual.
              ?entRole rdf:type owl:NamedIndividual.
              ?entRole rdfs:label ?entLabel.
                  FILTER regex(?entLabel, "''' +list_type+''' role")
              ?ent obo:RO_0000087 ?entRole
              ?ent afo:mentioned_in ?mention.
              ?mention afo:has_doi ?doi.
              FILTER regex(?doi, "10.1021/acscatal.7b00499.s001").
              OPTIONAL {
              			?ent rdfs:comment ?com.
          				FILTER NOT EXISTS {
          						FILTER regex(?com, "created automatically")
          						}
                        BIND(STR(?com) as ?entOtherName)   
                          }
            }                                      
        '''

    ent_list=list(default_world.sparql(sparqlstr))
    return ent_list

In [ ]:

reac_list=[['hydroformylation'], ['atmospheric hydroformylation']]
cat_list=[['Co'],['Rh']]
sup_list=[['SiO2']]

columns= ['eid',
          'doi',
          'pii',
          'pubmed_id',
          'title',
          'subtype',
          'subtypeDescription',
          'creator',
          'afid',
          'affilname',
          'affiliation_city',
          'affiliation_country',
          'author_count',
          'author_names',
          'author_ids',
          'author_afids',
          'coverDate',
          'coverDisplayDate',
          'publicationName',
          'issn',
          'source_id',
          'eIssn',
          'aggregationType',
          'volume',
          'issueIdentifier',
          'article_number',
          'pageRange',
          'description',
          'authkeywords',
          'citedby_count',
          'openaccess',
          'freetoread',
          'freetoreadLabel',
          'fund_acr',
          'fund_no',
          'fund_sponsor',
          'query']
df_all = pd.DataFrame(columns=columns)
results=[]
for r in reac_list:
    for cat in cat_list:
        for sup in sup_list:
            query = 'TITLE-ABS-KEY("{}"AND"{}"AND"{}")'.format(r[0],cat[0],sup[0])
            result = ScopusSearch(query, view='STANDARD',verbose=True)
            results.append(result)
            df = pd.DataFrame(pd.DataFrame(result.results))
            df['query'] = [query for _ in range(result.get_results_size())]
            df_all = pd.concat([df_all, df], axis=0, ignore_index=True)

distinct_df=df_all.drop_duplicates(['eid']).reset_index(drop=True)

distinct_df['coverDate']=pd.to_datetime(distinct_df['coverDate'])

In [ ]:
#filter found publication according the date
start_date = '2019-01-01' #or None
end_date = '2023-12-31' 
if start_date!= None:
    filtered_df = distinct_df[(distinct_df['coverDate'] >= start_date) & (distinct_df['coverDate'] <= end_date)]
else:
    filtered_df = distinct_df

onto_doi_list=list(default_world.sparql('''
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX afo: <http://purl.allotrope.org/voc/afo/merged/REC/2023/03/merged-without-qudt-and-inferred#>
PREFIX new:  <http://www.semanticweb.org/chern/ontologies/2023/10/new_onto.owl#>
SELECT ?doi
WHERE{
          ?publication rdf:type owl:NamedIndividual.
          ?publication rdf:type ?type.
          ?type rdfs:subClassOf* afo:publication.
          ?publication afo:has_doi ?doi
        }'''))
    
idx_none=[]
for row in filtered_df.itertuples():
    if row.doi == None:
        idx_none.append(row.Index)
filtered_df_doi= filtered_df.drop(index=idx_none)

query_doi_list=list(filtered_df['doi'])
idx_copy=[]
for d in onto_doi_list:
    if d[0] in query_doi_list:
        idx_copy.extend(list(filtered_df_doi.query('doi == "{}"'.format(d[0])).index))
filtered_df_doi_new = filtered_df.drop(index=idx_copy)


In [ ]:
#save from scopus quieried publication information in excel
with pd.ExcelWriter('output.xlsx', engine='xlsxwriter') as writer: #install xlsxwriter with pip
    # Write the first dataframe to a sheet named 'all'
    df_all.to_excel(writer, sheet_name='all', index=False)

    # Write the second dataframe to a sheet named 'distinct_eid'
    distinct_df.to_excel(writer, sheet_name='distinct_eid', index=False)
    
    #Write the third dataframe to a sheet named 'filtered_distinct'
    filtered_df.to_excel(writer, sheet_name='filtered_distinct', index=False)
    
    #Write the fourth dataframe to a sheet named 'filtered_with doi'
    filtered_df_doi.to_excel(writer, sheet_name='filtered_with_doi', index=False)
    
    #Write the fourth dataframe to a sheet named 'filtered_new_doi'
    filtered_df_doi_new.to_excel(writer, sheet_name='filtered_new_doi', index=False)